# Telecomm EDA project

### Import Libraries

In [1]:
!pip install folium
!pip install geopy

In [2]:
import numpy as np
import pandas as pd
import folium as folium
from geopy.geocoders import Nominatim

%matplotlib inline

# Cleanup

### Turn the CSV into a PD Dataframe

In [3]:
df = pd.read_csv("datasets/metadata.csv").drop(["Cell Cgi"], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10476 entries, 0 to 10475
Data columns (total 6 columns):
Cell Tower Location     10476 non-null object
Comm Identifier         1374 non-null object
Comm Timedate String    10476 non-null object
Comm Type               10476 non-null object
Latitude                10476 non-null float64
Longitude               10476 non-null float64
dtypes: float64(2), object(4)
memory usage: 491.1+ KB


### Turn Datetime String into a numpy Datetime Object

In [4]:
df["Datetime"] = pd.to_datetime(df["Comm Timedate String"], infer_datetime_format=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10476 entries, 0 to 10475
Data columns (total 7 columns):
Cell Tower Location     10476 non-null object
Comm Identifier         1374 non-null object
Comm Timedate String    10476 non-null object
Comm Type               10476 non-null object
Latitude                10476 non-null float64
Longitude               10476 non-null float64
Datetime                10476 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 573.0+ KB


### Make day of week column

In [5]:
df['Day of Week'] = df['Datetime'].dt.weekday_name

### Make the coordinates Folium Friendly :)

In [6]:
df['Location']=list(zip(df.Latitude,df.Longitude))

### Cleanup and reorganize DataFrame

In [7]:
df = df[['Datetime', 'Day of Week', 'Comm Identifier', 'Comm Type', 'Cell Tower Location', 'Location']]
df.head()

,Datetime,Day of Week,Comm Identifier,Comm Type,Cell Tower Location,Location
0,2014-04-01 09:40:00,Tuesday,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
1,2014-04-01 09:42:00,Tuesday,62157ccf2910019ffd915b11fa037243b75c1624,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
2,2014-04-01 13:13:00,Tuesday,c8f92bd0f4e6fb45ed7fce96fc831b283db2b642,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
3,2014-04-01 13:13:00,Tuesday,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
4,2014-04-01 17:27:00,Tuesday,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"


### Shorten Comm Identifier

In [8]:
df["Comm Identifier"] = df["Comm Identifier"].str[:10]

df.head()

,Datetime,Day of Week,Comm Identifier,Comm Type,Cell Tower Location,Location
0,2014-04-01 09:40:00,Tuesday,f1a6836c0b,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
1,2014-04-01 09:42:00,Tuesday,62157ccf29,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
2,2014-04-01 13:13:00,Tuesday,c8f92bd0f4,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
3,2014-04-01 13:13:00,Tuesday,f1a6836c0b,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
4,2014-04-01 17:27:00,Tuesday,f1a6836c0b,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"


# EDA

### Where this individual spends the most time.

They can usually be found in Sydney Australia.

- spends both weekend and weekday time near Amazonia Martial Arts, his top location overall.  this is likely his home
- spends a lot of weekday time at IFGF Sydney (a church), maybe volunteering?
- parks at City Parking in Haymarket on weekdays, rarely on weekends.  this is probably near where he works

In [9]:
df['Location'].dropna().value_counts().head(10)

(-33.78815, 151.26654)         4301
(-33.88417103, 151.20235)      1084
(-42.84338, 147.29569)          723
(-33.89293336, 151.2022962)     712
(-33.88032891, 151.2056904)     563
(-42.85984, 147.29215)          501
(-33.779333, 151.276901)        465
(-33.79661, 151.27756)          454
(-33.796679, 151.285293)        231
(-42.85307, 147.31532)          197
Name: Location, dtype: int64

### 4301
-33.788150, 151.266540 = Amazonia Martial Arts Club (Manly Beach 

### 1084
-33.884171, 151.202350 = IFGF Sydney (a church) (Haymarket, Sydney)

### 723
-42.843380, 147.295690 = Shopping Mall, Intercity Cycleway (Tasmania)

### 712
-33.892933, 151.202296 = Office Building outside fire station (Redfern, Sydney)

### 563
-33.880329, 151.205690 = City Parking (Haymarket, Sydney)

There are four distinct periods of time where the individual goes to Tasmania: 
- 5/2/2014 to 5/4/2014
- 8/10/2014 to 8/12/2014
- 11/12/2014 to 11/15/2014
- 12/20/2014 to 12/26/2014

### Weekdays vs Weekends

In [10]:
# build a mask to select days 5 and 6 (saturday, sunday)
df.loc[~df.Datetime.dt.weekday.isin([5,6])]

,Datetime,Day of Week,Comm Identifier,Comm Type,Cell Tower Location,Location
0,2014-04-01 09:40:00,Tuesday,f1a6836c0b,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
1,2014-04-01 09:42:00,Tuesday,62157ccf29,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
2,2014-04-01 13:13:00,Tuesday,c8f92bd0f4,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
3,2014-04-01 13:13:00,Tuesday,f1a6836c0b,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
4,2014-04-01 17:27:00,Tuesday,f1a6836c0b,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
5,2014-04-01 17:36:00,Tuesday,6bbc17070a,Phone,CHIPPENDALE,"(-33.88417103, 151.20235)"
6,2014-04-01 17:40:00,Tuesday,6bbc17070a,Phone,CHIPPENDALE,"(-33.88417103, 151.20235)"
7,2014-04-02 19:18:00,Wednesday,7cb96eadd3,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
8,2014-04-03 14:35:00,Thursday,de40c5c1f9,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
9,2014-04-03 14:36:00,Thursday,66f32c1163,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"


In [11]:
# build a mask to select weekdays
df.loc[df.Datetime.dt.weekday.isin([5,6])]

,Datetime,Day of Week,Comm Identifier,Comm Type,Cell Tower Location,Location
22,2014-04-05 15:10:00,Saturday,dc6774d10e,Phone,MANLY #,"(-33.796679, 151.285293)"
50,2014-04-12 08:16:00,Saturday,0767e517fa,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
51,2014-04-13 01:42:00,Sunday,6ce4ad58c4,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
52,2014-04-13 10:58:00,Sunday,6bbc17070a,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
53,2014-04-13 11:20:00,Sunday,cad22015a5,SMS,REDFERN TE,"(-33.89293336, 151.2022962)"
54,2014-04-13 17:44:00,Sunday,6bbc17070a,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
91,2014-04-19 12:34:00,Saturday,a5834ee77b,Phone,FAIRLIGHT 137 SYDNEY RD,"(-33.79661, 151.27756)"
135,2014-04-26 11:11:00,Saturday,a804558e42,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
136,2014-04-26 11:47:00,Saturday,cd3b394668,SMS,REDFERN TE,"(-33.89293336, 151.2022962)"
137,2014-04-26 11:49:00,Saturday,cd3b394668,SMS,REDFERN TE,"(-33.89293336, 151.2022962)"


### Select comm type

In [12]:
# put which types you want to select in this list and select from it below
comm_types = ['Phone','SMS','Internet']

df[df['Comm Type'].isin(comm_types)]

,Datetime,Day of Week,Comm Identifier,Comm Type,Cell Tower Location,Location
0,2014-04-01 09:40:00,Tuesday,f1a6836c0b,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
1,2014-04-01 09:42:00,Tuesday,62157ccf29,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
2,2014-04-01 13:13:00,Tuesday,c8f92bd0f4,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
3,2014-04-01 13:13:00,Tuesday,f1a6836c0b,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
4,2014-04-01 17:27:00,Tuesday,f1a6836c0b,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
5,2014-04-01 17:36:00,Tuesday,6bbc17070a,Phone,CHIPPENDALE,"(-33.88417103, 151.20235)"
6,2014-04-01 17:40:00,Tuesday,6bbc17070a,Phone,CHIPPENDALE,"(-33.88417103, 151.20235)"
7,2014-04-02 19:18:00,Wednesday,7cb96eadd3,Phone,REDFERN TE,"(-33.89293336, 151.2022962)"
8,2014-04-03 14:35:00,Thursday,de40c5c1f9,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"
9,2014-04-03 14:36:00,Thursday,66f32c1163,Phone,HAYMARKET #,"(-33.88032891, 151.2056904)"


### Look at tower locations and build addresses

In [13]:
# make a dictionary to store addresses
address={}

# look up addresses and store them in the dictionary
geolocator = Nominatim()
for i in df.Location.unique() :
    location= geolocator.reverse(i)
    address[i]=location.address

In [14]:
# clean up the address
addressl={}
for k,v in address.items():
    fields=list(v.split(','))
    addressl[k]=fields

In [15]:
addressl

{(-42.884809999999995, 147.32748000000001): [u"Domino's",
  u' Macquarie Street',
  u' Hobart',
  u' Tasmania',
  u' 7000',
  u' Australia'],
 (-42.88194, 147.32893000000001): [u'D\xf4me',
  u' 29',
  u' Elizabeth Street',
  u' Hobart',
  u' Tasmania',
  u' 7000',
  u' Australia'],
 (-42.881009999999996, 147.33346): [u'Hotel Grand Chancellor Hobart',
  u' 1',
  u' Davey Street',
  u' Hobart',
  u' Tasmania',
  u' 7000',
  u' Australia'],
 (-42.880290000000002, 147.32808): [u'Dumpling World',
  u' Liverpool Street',
  u' Hobart',
  u' Tasmania',
  u' 7000',
  u' Australia'],
 (-42.874569999999999, 147.31788): [u'Burnett St',
  u' Burnett Street',
  u' North Hobart',
  u' Tasmania',
  u' 7000',
  u' Australia'],
 (-42.860599999999998, 147.45419999999999): [u'Mount Rumney Road',
  u' Acton Park',
  u' Mount Rumney',
  u' Tasmania',
  u' 7019',
  u' Australia'],
 (-42.859840000000005, 147.29214999999999): [u'William Cooper Drive',
  u' New Town',
  u' Tasmania',
  u' 7008',
  u' Australia'

In [16]:
# make a field for each part of the address to sort and show

# df['address']=df.Location.map(lambda x: address[x])
df['Place']=df.Location.map(lambda x: addressl[x][0])
df['Town']=df.Location.map(lambda x: addressl[x][-4])
df['Region']=df.Location.map(lambda x: addressl[x][-3])
df['Zip']=df.Location.map(lambda x: addressl[x][-2])
df['Country']=df.Location.map(lambda x: addressl[x][-1])

In [17]:
# what unique regions has he/she visited?

df['Region'].unique()

array([u' NSW', u' Tasmania', u' Victoria', u' Hume'], dtype=object)

### Select hour groupings (night, work, after work)

In [18]:
# from midnight to 8am
df[df.Datetime.dt.hour < 8]

# select top locations for this time period
df[df.Datetime.dt.hour < 8]['Location'].value_counts().head()

(-33.78815, 151.26654)         743
(-33.88417103, 151.20235)      128
(-33.79661, 151.27756)          48
(-33.779333, 151.276901)        45
(-33.88032891, 151.2056904)     40
Name: Location, dtype: int64

In [19]:
# from 8am-5pm
df[(df.Datetime.dt.hour > 7) & (df.Datetime.dt.hour < 17)]

# select top locations for this time period
df[(df.Datetime.dt.hour > 7) & (df.Datetime.dt.hour < 17)]['Location'].value_counts().head()

(-33.78815, 151.26654)         1758
(-33.88417103, 151.20235)       685
(-33.88032891, 151.2056904)     407
(-42.84338, 147.29569)          378
(-33.89293336, 151.2022962)     241
Name: Location, dtype: int64

In [20]:
# from 5pm-midnight
df[(df.Datetime.dt.hour > 16)]

# select top locations for this time period
df[(df.Datetime.dt.hour > 16)]['Location'].value_counts().head()

(-33.78815, 151.26654)         1800
(-33.89293336, 151.2022962)     450
(-42.84338, 147.29569)          338
(-33.88417103, 151.20235)       271
(-42.85984, 147.29215)          262
Name: Location, dtype: int64

### Plot on the map!

In [21]:
# access each location by index, and then index inside the tuple
display(df.Location.iloc[5][0])
display(df.Location.iloc[5][1])

-33.884171029999997

151.20235

In [22]:
# plotting code here is just an example - I'm not plotting every combination above
# in this case, we're just plotting all 70 unique locations

map_location = folium.Map(location=[-35.136188, 150.612841], zoom_start = 5)

for x in df.Location.unique():
    folium.Marker((x[0], x[1]), popup='It worked!').add_to(map_location)
map_location

### add these to a heatmap showing frequency at locations

In [23]:
from folium.plugins import HeatMap, HeatMapWithTime

# initialize heatmap, add center point
heatmap_location = folium.Map(location=[-35.136188, 150.612841], zoom_start = 5)

# add the location column to map
HeatMap(df.Location).add_to(heatmap_location)

In [26]:
heatmap_location

### Most Frequent Contacts

In [25]:
df['Comm Identifier'].dropna().value_counts().head(10)

bc0b018604    219
12e3d1b0c9    146
91aba4a113    144
a24a4646d0    133
6bbc17070a     83
a804558e42     62
cd3b394668     56
c22670da93     44
70e1f163d8     39
c521537546     31
Name: Comm Identifier, dtype: int64